In [1]:
import boto3
import logging 
from botocore.exceptions import ClientError
import matplotlib.pyplot as plt 

import pandas as pd 
import io

## Load data 

In [2]:
# Function to read the parquet file as pandas dataframe 
def open_S3_file_as_df(bucket_name, file_name):
    """Open a S3 parquet file from bucket and filename and return the parquet as pandas dataframe
    :param bucket_name: Bucket name
    :param file_name: Specific file name to open
    :return: body of the file as a string
    """
    try: 
        s3 = boto3.resource('s3')
        object = s3.Object(bucket_name, file_name)
        body = object.get()['Body'].read()
        df = pd.read_parquet(io.BytesIO(body))
        print(f'Loading {file_name} from {bucket_name} to pandas dataframe')
        return df
    except ClientError as e:
        logging.error(e)
        return e
file_name = "records.parquet"
bucket_name = "webpresence-geocore-geojson-to-parquet-dev"
bucket_name_nlp='nlp-data-preprocessing'
df_raw = open_S3_file_as_df(bucket_name, 'records.parquet')
df_clean = open_S3_file_as_df(bucket_name_nlp, 'Processed_records.parquet')


Loading records.parquet from webpresence-geocore-geojson-to-parquet-dev to pandas dataframe
Loading Processed_records.parquet from nlp-data-preprocessing to pandas dataframe


## Simple cleaning before visualization 
Here, we will start with the raw dataset.
* Duplicate rows have been removed.
* Columns with more than 50% missing values have been removed, leaving us with 43 columns.
* Remaining missing values in categorical columns have been filled with 'Unknown'.
* The single numerical column ('features_popularity') did not have any missing values, so no filling was needed there.
* Remove columns with French text 
* Subset to columns with meaningful information 

In [3]:
# Display the first few rows and the general information about the data
df_raw_info = df_raw.info()
df_raw_head = df_raw.head()

df_raw_info, df_raw_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7343 entries, 0 to 7342
Data columns (total 68 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   features_type                                                  7343 non-null   string
 1   features_geometry_type                                         7343 non-null   string
 2   features_geometry_coordinates                                  7343 non-null   string
 3   features_properties_id                                         7343 non-null   object
 4   features_properties_title_en                                   7342 non-null   string
 5   features_properties_title_fr                                   7342 non-null   string
 6   features_properties_description_en                             7343 non-null   string
 7   features_properties_description_fr                             7343 n

(None,
   features_type features_geometry_type  \
 0       Feature                Polygon   
 1       Feature                Polygon   
 2       Feature                Polygon   
 3       Feature                Polygon   
 4       Feature                Polygon   
 
                        features_geometry_coordinates  \
 0  [[[-141.003, 41.6755], [-52.6174, 41.6755], [-...   
 1  [[[-142, 41], [-52, 41], [-52, 84], [-142, 84]...   
 2  [[[-141.003, 41.6755], [-52.6174, 41.6755], [-...   
 3  [[[-104.75571511, 50.42392886], [-104.56356008...   
 4  [[[-141.003, 41.6755], [-52.6174, 41.6755], [-...   
 
                  features_properties_id  \
 0  000183ed-8864-42f0-ae43-c4313a860720   
 1  7f245e4d-76c2-4caa-951a-45d1d2051333   
 2  085024ac-5a48-427a-a2ea-d62af73f2142   
 3  03ccfb5c-a06e-43e3-80fd-09d4f8f69703   
 4  488faf70-b50b-4749-ac1c-a1fd44e06f11   
 
                         features_properties_title_en  \
 0  Principal Mineral Areas, Producing Mines, and ...   
 1       

In [4]:
# Remove duplicate rows
df_raw.drop_duplicates(inplace=True)

# Remove columns with more than 50% missing values
threshold = len(df_raw) * 0.5
df_raw.dropna(thresh=threshold, axis=1, inplace=True)

# Fill missing values
# For categorical columns fill with 'Unknown'
categorical_cols = df_raw.select_dtypes(include=['object']).columns
df_raw[categorical_cols] = df_raw[categorical_cols].fillna('Unknown')

# For numerical columns fill with median
numerical_cols = df_raw.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    median_value = df_raw[col].median()
    df_raw[col].fillna(median_value, inplace=True)

# Identify the columns that contain "_fr" in their names
cols_fr = [col for col in df_raw.columns if '_fr' in col]

# Drop these columns
df_raw.drop(columns=cols_fr, inplace=True)

# Check the result
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7343 entries, 0 to 7342
Data columns (total 38 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   features_type                                7343 non-null   string
 1   features_geometry_type                       7343 non-null   string
 2   features_geometry_coordinates                7343 non-null   string
 3   features_properties_id                       7343 non-null   object
 4   features_properties_title_en                 7342 non-null   string
 5   features_properties_description_en           7343 non-null   string
 6   features_properties_keywords_en              6194 non-null   string
 7   features_properties_topicCategory            7343 non-null   string
 8   features_properties_date_published_text      7322 non-null   string
 9   features_properties_date_published_date      7321 non-null   string
 10  features_pro

In [5]:
# Subset the data to include only the columns of interest
cols =  ['features_properties_id','features_geometry_coordinates', 'features_properties_title_en','features_properties_description_en','features_properties_keywords_en',
         'features_properties_topicCategory', 'features_properties_date_published_date', 'features_properties_date_created_date', 'features_properties_temporalExtent_begin',
         'features_properties_temporalExtent_end','features_properties_spatialRepresentation','features_properties_type','features_properties_geometry','features_properties_status',
         'features_properties_metadataStandard_en','features_properties_metadataStandardVersion','features_properties_graphicOverview', 'features_properties_locale_language', 
         'features_properties_language','features_popularity']
df = df_raw[cols]
print(df.shape)

(7343, 20)


In [6]:
df.to_csv('df_raw_cleaned.csv', index=False)

## Visualization 
given the nature of your dataset, which is mostly categorical and textual, we have limited options for numerical or continuous data visualization.

* A pie chart of the different categories in the features_properties_spatialRepresentation column.
* A pie chart of the different categories in the features_properties_topicCategory column.
* A pie chart of the different categories in the features_properties_type column.
* A pie chart of the different categories in the features_properties_status column.
* A piechart features_properties_metadataStandard_en or features_properties_metadataStandardVersion
* A pie chart of the different categories in the features_properties_locale_language column.
* A histogram of the features_popularity column.
* ?? [Word visual cloud](https://thecleverprogrammer.com/2021/11/11/word-cloud-from-a-pandas-dataframe-in-python/#:~:text=A%20word%20cloud%20is%20a,based%20on%20natural%20language%20processing) of features_properties_title_en, features_properties_description_en, features_properties_keywords_en


* ?? Time range from features_properties_temporalExtent_begin and features_properties_temporalExtent_end 
* A pie chart of the different categories in the features_properties_graphicOverview (Y and N)



In [7]:
print(df_raw['features_properties_status'].unique())
print(df_raw['features_properties_spatialRepresentation'].unique())


<StringArray>
[                 'completed; complété',
 'historicalArchive; archiveHistorique',
                  'onGoing; enContinue',
          'Not Available; Indisponible',
                    'proposed; proposé',
                    'planned; planifié',
       'underDevelopment; enProduction',
                              'unknown']
Length: 8, dtype: string
<StringArray>
[            'vector; vecteur',                'grid; grille',
 'Not Available; Indisponible',                          <NA>,
       'textTable; texteTable',                'video; vidéo']
Length: 6, dtype: string


In [8]:
# pip install dash dash_core_components dash_html_components plotly pandas wordcloud
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS
from PIL import Image
import io
import base64

import nltk
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import word_tokenize   # module for tokenizing strings                            # for string operations
import re

C:\Users\xcai\AppData\Local\Temp\ipykernel_11048\3253786129.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\xcai\AppData\Local\Temp\ipykernel_11048\3253786129.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


For the interactive dashboard, we will include the following features. 
1.  A section showing the overall description of the data 
2. A section showing the most common topic categories, statuses using bar chars,  showing the spatial representations, feature types, , metadata standards, locale languages, and languages using pie chart 
3. Two bar charts showing the years in which the most features were published and created, with a slider or input box to adjust the range of years displayed.
4. A histogram showing the distribution of the popularity scores, with sliders or input boxes to adjust the range of scores displayed.
5. A word visual cloud to show the condatinate of of features_properties_title_en, features_properties_description_en, features_properties_keywords_en


In [9]:
# Create dataframes for the visuals
df_topic = df['features_properties_topicCategory'].value_counts().reset_index().head(10)
df_spatial = df['features_properties_spatialRepresentation'].value_counts().reset_index()
df_status = df['features_properties_status'].value_counts().reset_index().head(10)
df_type = df['features_properties_type'].value_counts().reset_index().head(10)
df_standard = df['features_properties_metadataStandard_en'].value_counts().reset_index().head(10)
#TODO language
#TODO popularity slider 
#TODO temporal extent slider
df_language = df['features_properties_language'].value_counts().reset_index().head(10)

In [10]:
# Define a function to extract year from date strings
def extract_year(date_str):
    if isinstance(date_str, str):
        return date_str.split('-')[0]
    return np.nan
# Apply the function to the 'features_properties_date_published_date' and 'features_properties_date_created_date' columns
df['year_published'] = df['features_properties_date_published_date'].apply(extract_year)
df['year_created'] = df['features_properties_date_created_date'].apply(extract_year)
df_year_published = df['year_published'].value_counts().reset_index().head(10)
df_year_created = df['year_created'].value_counts().reset_index().head(10)

C:\Users\xcai\AppData\Local\Temp\ipykernel_11048\866143872.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_published'] = df['features_properties_date_published_date'].apply(extract_year)
C:\Users\xcai\AppData\Local\Temp\ipykernel_11048\866143872.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_created'] = df['features_properties_date_created_date'].apply(extract_year)


In [11]:
# Create the visuals
fig_topic = px.bar(df_topic, x='index', y='features_properties_topicCategory', title='Topic 10 Categories')
fig_status = px.bar(df_status, x='index', y='features_properties_status', title='Top 10 Status')
fig_spatial = px.pie(df_spatial, values='features_properties_spatialRepresentation', names='index', title='Spatial Representations')
fig_type = px.pie(df_type, values='features_properties_type', names='index', title='Top 10 Feature  Types')
fig_standard = px.pie(df_standard, values='features_properties_metadataStandard_en', names='index', title='Metadata Standards of the records')
fig_language= px.pie(df_language, values='features_properties_language', names='index', title='Languages of the records')
fig_year_published = px.bar(df_year_published, x='index', y='year_published', title='Top 10 Years of Published Date')
fig_year_created = px.bar(df_year_created, x='index', y='year_created', title='Top 10 Years of Created Date')

In [14]:
# Function to create wordcloud 
def create_word_cloud(df):
    text = " ".join(i for i in df)
    print('Word count for all the titles are {}'. format(len(text)))
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    return wordcloud.to_image()


In [15]:
word_cloud_image = create_word_cloud(df_clean['features_properties_title_en'])  
buffered = io.BytesIO()
word_cloud_image.save(buffered, format="PNG")
img_str = base64.b64encode(buffered.getvalue()).decode()


Word count for all the titles are 395152


In [22]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [23]:
# Define the layout
app.layout = html.Div(children=[
    html.H1(children='Dashboard'),

    html.Div(children='''
        This dashboard shows the overall description of the data.
    '''),

    dcc.Graph(figure=fig_topic),
    dcc.Graph(figure=fig_status),
    dcc.Graph(figure=fig_spatial),
    dcc.Graph(figure=fig_type),
    dcc.Graph(figure=fig_standard),
    dcc.Graph(figure=fig_language),
    dcc.Graph(figure=fig_year_published),
    dcc.Graph(figure=fig_year_created),
    
    
    html.Div(children=[
        html.Img(src='data:image/png;base64,{}'.format(img_str))
    ])
])

In [24]:
# Run the app
app.run_server(debug=True)